# **Laboratory 2 - PID Design via Loop shaping**
In this laboratory session, you will use linear control theory concepts and get your hands on a concrete example: the cart-pole. 
More especially, you will design a controller for the cart-pole such that it is robust to a disturbance modeled as a push on the cart.

## **Instructions**

**Deliverable.**
You should answer the questions of this lab in a short `.pdf` report.

**Deadline.**
The submission deadline is on the **14th of March 2025 at 11:59PM**.

**Submission.**
Submit your report on [Gradescope](https://www.gradescope.com/) by logging in with your account `@student.uliege.be`. Don't forget to assign the corresponding pages to the different questions when submitting.
If the course does not appear in your dashboard on Gradescope, contact us on [Ed Discussion](https://edstem.org/us/dashboard) quickly. If you are not familiar with submitting your work to Gradescope, you will find all the necessary information in the [online help](https://help.gradescope.com/article/ccbpppziu9-student-submit-work).

**Collaboration policy.**
You can discuss the assignment with other students, but *you must write your own solutions, and write and run your own code*. Copying someone else's solution, or just making trivial changes to avoid copying verbatim, is not acceptable.


**You may just `Run All` and then you can start.**

In [ ]:
from ipywidgets import *
from IPython.display import display
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# import logging
plt.close("all")

COLAB=False
try :
    %matplotlib widget
except:
    mpl.use('nbagg')
    COLAB=True

mpl.rcParams['lines.linewidth'] = 2.5
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.labelpad'] = 10
plt.rcParams['axes.titlepad'] = 15
plt.rcParams['axes.titlesize'] = 'large'
plt.rcParams['axes.labelsize'] = 'large'
plt.rcParams['axes.labelweight'] = 'bold'

try :
    import control
except:
    print("/!\ python-control package is not installed" )
    !pip install control
    import control

gravity = 9.81
mass_cart = 1.0
mass_pole = 0.1
length = 0.5
def cartpole_update(t, x, u, params):
    g = 9.81  # gravity
    mc = 1.0  # mass of the cart
    mp = 0.1  # mass of the pole
    l = 0.5   # length of the pole

    # states
    _, x_dot, theta, theta_dot = x
    F = u[0] + u[1]

    # equations of motion
    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)
    divisor = mc + mp*sin_theta**2

    x_ddot = (F + mp*sin_theta*(l*theta_dot**2 + g*cos_theta))/(divisor)
    theta_ddot = (-F*cos_theta -mp*l*theta**2 * sin_theta*cos_theta - (mc+mp)*g*sin_theta)/(l*divisor)

    return [x_dot, x_ddot, theta_dot, theta_ddot]

def cartpole_output(t, x, u, params):
    return x[2] + u[-1]

SYSTEM = control.nlsys(cartpole_update, cartpole_output, inputs=('u', 'v', 'w'), outputs=(r'$\theta$'), 
                         states=('x', 'x_dot', r'$\theta$', 'theta_dot'), name='cartpole')
TF_LIN_SYSTEM = control.tf([1], [length*mass_cart, 0, -gravity*(mass_pole+mass_cart)])

def disturbance(time):
    signal = np.where(time < 0.25, 100.0*np.sin(4*np.pi*time), np.zeros_like(time))
    return signal

def gof_plot_func(*args, ax=None,**kwargs):
    cplt = control.gangof4_plot(*args, **kwargs, ax=ax)
    ax_titles = ['$T(s)$', '$PS(s)$', '$CS(s)$', '$S(s)$']
    ax = ax if ax is not None else cplt.axes
    for i, axis in enumerate(ax.flatten()):
        axis.set_title(ax_titles[i])
        if not i%2:
            axis.set_ylabel('Magnitude [dB]')
            
    return cplt

class OutPlot:
    def __init__(self, **kwargs):
        self.output = Output()
        self._init_plot(**kwargs)

    def __call__(self, *args, **kwds):
        self.output.clear_output(wait=True)
        with self.output:
            self._plot(*args, **kwds)

    def _plot(self, *args, **kwds):
        pass

    def _init_plot(self, *args, **kwds):
        pass

class ResponsePlot(OutPlot):
    def _init_plot(self, fig_name="all", title=None, tau=5e-2, seed=42):

        np.random.seed(seed)
        self.output.layout.height = '100%'
        self.output.layout.width = '50%'
        plt.close(fig_name)
        self.figure = plt.figure(fig_name, figsize=(12,5),
                                 dpi=100, clear=True)
        self.figure.canvas.header_visible = False
        self.ax = self.figure.add_subplot(1,1,1)
        self.ax.set_xlabel("Time [s]")
        self.ax.set_ylabel("Amplitude [rad]")
        self.ax.set_title(title)

        self.final_T = 15
        self.ax.hlines(np.pi, 0, self.final_T, color='r', linestyle='--', zorder=10)
        self.lines, = self.ax.plot([], [], color='blue')
        self.ax.legend(['Reference', '$\Theta$'])
        self.lowpass = control.zpk([], [-1/tau]*3, (-1/tau)**3, dt=0)
        self.highpass = 1 - self.lowpass

        self.time = np.linspace(0, self.final_T, 5000)
        white_noise = control.white_noise(self.time, np.array([[1e-4, 0], [0, 0.1]]))
        self.disturbance = control.forced_response(self.lowpass, T=self.time, U=white_noise[1])
        self.noise = control.forced_response(self.highpass, T=self.time, U=white_noise[0])

        self.parameters = dict(kp=1, ki=0, kd=0, noise=False, disturbance=False, deviation=np.pi/12)

    def _on_change(self, **kwargs):
        self.parameters.update(kwargs)

    def _on_click(self, button):
        self(**self.parameters)
        
    def _plot(self, kp, ki, kd, 
              noise=False, disturbance=False,
              deviation=2*np.pi/12):
        t = self.time
        pi_controller = control.tf([kp], [1]) + control.tf([ki], [1, 0]) + control.tf([kd*1, 0], [1e-3, 1])
        pi_controller.update_names(name='pi_controller')
        open_loop = control.interconnect([SYSTEM, pi_controller], 
                                         connections=[['cartpole.u', 'pi_controller.y'],], 
                                         inplist=['pi_controller.u', 'cartpole.v', 'cartpole.w'], 
                                         outlist=[r'cartpole.$\theta$'],
                                         name='open_loop')
        
        sum_junc = control.summing_junction(2, 1, name='sum_junc')
        cartpole_pi = control.interconnect([open_loop, sum_junc],
                                           connections=[['sum_junc.u[1]', '-open_loop.y[0]'],
                                                        ['open_loop.u[0]','sum_junc.y[0]']],
                                           inplist=['sum_junc.u[0]', 'open_loop.u[1]', 'open_loop.u[2]'],
                                            outlist=['open_loop.y[0]'],
                                            name='closed_loop')

        inputs = np.zeros((3, len(t)))
        inputs[0]+=np.pi
        if noise:
            inputs[2] = 1.0 * self.noise.y
        if disturbance:
            inputs[1] = 1.0 * self.disturbance.y

        print("Computing ...", end='\r')
        states = control.input_output_response(cartpole_pi, U=inputs,
                                       X0=[0,0,np.pi - deviation,0], T=t, squeeze=True)
        
        # Transform the dictionary into printable key-value pairs
        step_info = control.step_info(states.outputs, T=t)
        print(*[f"{key}: {value:.05f}" for key, value in step_info.items()], sep='\n')
        self.lines.remove()
        self.lines, = self.ax.plot(t, states.states[2], color='blue')
        min_ax, max_ax = min(min(states.y[0]), np.pi), max(max(states.y[0]), np.pi)
        self.ax.set_ylim([min_ax - 0.1, max_ax + 0.1])
        display(self.figure)

class FullPlot(OutPlot):
    def _init_plot(self, fig_name="all", title=None):
        self.output.layout.height = '100%'
        self.output.layout.width = '100%'

        plt.close(fig_name)
        self.figure = plt.figure(fig_name, figsize=(12, 10), dpi=100)
        self.axes = self.figure.subplots(2, 2)
        self.figure.subplots_adjust(hspace=0.5)
        self.parameters = dict(k=1, zlead=0, plead=0)
        self.plant = TF_LIN_SYSTEM.copy()
        self.plant.update_names(name='plant')
        self.time = np.linspace(0, 15, 5000)

    def _on_change(self, **kwargs):
        self.parameters.update(kwargs)

    def _on_click(self, button):
        self(**self.parameters)
        
    def _plot(self, k, zlead, plead):
        t = self.time
        controller = k * control.tf([1, zlead], [1, plead])
        controller.update_names(name='control')
        open_loop = controller * self.plant
        open_loop.update_names(name='open_loop')
        
        sum_junc = control.summing_junction(2, 1, name='sum_jct')
        closed_loop = control.interconnect([controller, self.plant, sum_junc],
                                            [['control.u[0]', '-plant.y[0]'],
                                            ['plant.u[0]', 'sum_jct.y[0]'],
                                            ['sum_jct.u[0]', 'control.y[0]']],
                                            inplist=['sum_jct.u[1]'],
                                            outlist=['plant.y'], name='closed_loop', debug=False)

        inputs = disturbance(t)
        print("Computing ...", end='\r')
        for ax in self.axes.flatten():
            ax.clear()
        
        #Bode plots of the system
        control.bode_plot(open_loop, display_margins='overlay', dB=True, ax=self.axes[:,0])
        self.axes[0, 0].set_title('Bode plots')

        # Nyquist plot of the system
        plt.figure(self.figure.number)
        plt.sca(self.axes[0, 1])
        control.nyquist_plot(open_loop, unit_circle=True, ax=self.axes[0, 1])
        self.axes[0, 1].set_title('Nyquist plot')

        # Step response of the system
        states = control.input_output_response(closed_loop, U=inputs, T=t, squeeze=True)
        self.axes[1, 1].plot(t, states.y[0])
        self.axes[1, 1].set(title='Time response to $d(t)$', xlabel='Time [s]', ylabel='Angle deviation [rad]')
        print('')
        display(self.figure)

class RespPlot(OutPlot):
    def _init_plot(self, fig_name="all", title=None):
        self.output.layout.height = '100%'
        self.output.layout.width = '100%'

        plt.close(fig_name)
        self.figure = plt.figure(fig_name, figsize=(12, 5), dpi=100)
        self.ax = self.figure.add_subplot(1,1,1)
        self.ax.set_xlabel("Time [s]")
        self.ax.set_ylabel("Angle deviation [rad]")
        self.ax.set_title(title)

        self.parameters = dict(k=1, zlead=0, plead=0, zpi=0)
        self.plant = TF_LIN_SYSTEM.copy()
        self.plant.update_names(name='plant')
        self.time = np.linspace(0, 15, 5000)

    def _on_change(self, **kwargs):
        self.parameters.update(kwargs)

    def _on_click(self, button):
        self(**self.parameters)
        
    def _plot(self, k, zlead, plead, zpi):
        plt.figure(self.figure.number)
        t = self.time
        controller = k * control.tf([1, zlead], [1, plead]) * control.tf([1, zpi], [1, 0])
        controller.update_names(name='control')
        open_loop = controller * self.plant
        open_loop.update_names(name='open_loop')
        
        sum_junc = control.summing_junction(2, 1, name='sum_jct')
        closed_loop = control.interconnect([controller, self.plant, sum_junc],
                                            [['control.u[0]', '-plant.y[0]'],
                                            ['plant.u[0]', 'sum_jct.y[0]'],
                                            ['sum_jct.u[0]', 'control.y[0]']],
                                            inplist=['sum_jct.u[1]'],
                                            outlist=['plant.y'], name='closed_loop', debug=False)

        inputs = disturbance(t)
        print("Computing ...", end='\r')
        self.ax.clear()
        # Step response of the system
        states = control.input_output_response(closed_loop, U=inputs, T=t, squeeze=True)
        self.ax.plot(t, states.y[0])
        self.ax.set(title='Time response to $d(t)$', xlabel='Time [s]', ylabel='Angle deviation [rad]')
        print('')
        display(self.figure)

class GoFPlot(OutPlot):
    def _init_plot(self, fig_name="all", title=None):
        self.output.layout.height = '100%'
        self.output.layout.width = '100%'

        plt.close(fig_name)
        self.figure = plt.figure(fig_name, figsize=(12, 5), dpi=100)
        self.ax = self.figure.subplots(2, 2)
        self.figure.subplots_adjust(hspace=0.5)

        self.parameters = dict(k=1, zlead=0, plead=0, zpi=0)
        self.plant = TF_LIN_SYSTEM.copy()
        self.plant.update_names(name='plant')
        ax_titles = ['$T(s)$', '$PS(s)$', '$CS(s)$', '$S(s)$']
        for i, axis in enumerate(self.ax.flatten()):
            axis.set_title(ax_titles[i])

    def _on_change(self, **kwargs):
        self.parameters.update(kwargs)

    def _on_click(self, button):
        self(**self.parameters)
        
    def _plot(self, k, zlead, plead, zpi):
        plt.figure(self.figure.number)
        lead_control = k * control.tf([1, zlead], [1, plead])
        lead_pi_control = k * control.tf([1, zlead], [1, plead]) * control.tf([1, zpi], [1, 0])
        lead_control.update_names(name='Lead controller')
        lead_pi_control.update_names(name='Lead-PI controller')
        print("Computing ...", end='\r')
        
        [axis.clear() for axis in self.ax.flatten()]
        # Gang of Four
        omegas = np.logspace(-2, 2, 1000)
        gof_plot_func(self.plant, lead_control, ax=self.ax, omega=omegas, dB=True)
        gof_plot_func(self.plant, lead_pi_control, ax=self.ax, omega=omegas, linestyle='--', dB=True)
        print('')
        display(self.figure)

def create_buttons(kp=True, ki=True, kd=False, noise=False):
    param_list=[]
    if kp:
        param_list.append(BoundedFloatText(value=1, min=0, max=100, step=1, description='Kp'))
    if ki:
        param_list.append(BoundedFloatText(value=0, min=0, max=100, step=.1, description='Ki'))
    if kd:
        param_list.append(BoundedFloatText(value=0, min=0, max=100, step=.1, description='Kd'))
    if noise:
        param_list.append(Checkbox(value=False, description='Add measurement noise'))
        param_list.append(Checkbox(value=False, description='Add disturbance'))
    param_list.append(Button(description='Run simulation'))
    box = VBox(param_list, layout=Layout(flex_flow='column', align_items='center'))
    return box, param_list



## **Models**

#### **Cart-Pole**
Before designing our controller, we need a model for the cart-pole. We will focus on the angle of the pole $\theta$.
The system being non-linear, we will linearize it around the equilibrium point $\theta = \pi$. 
The parameters are the length of the pole $l$, the mass of the cart $m_c$, the mass of the pole $m_p$, and the gravity constant $g$. 
The input is the force $f_x$ applied to the cart, and the output is the deviation $\tilde \theta$ of the angle around the equilibrium point $\theta = \pi$ of the pole.

The transfer function of the linearized system is given by
\begin{equation*}
    P(s) = \frac{1}{l\cdot m_c \cdot s^2 - g\cdot (m_c + m_p)}    
\end{equation*}
where $l = 0.5$ m, $m_c = 1$ kg, $m_p = 0.1$ kg, and $g = 9.81$ m/s $^2$.

#### **Controller**
The force actuator of the cart will be controlled by a lead compensator and a PI block connected in series, so that the controller transfer function writes
$$C(s) = K \frac{s+z}{s+p}\frac{s+z_{PI}}{s}.$$
The lead compensator will be centered on the desired gain crossover frequency $\omega_c$ and its pole and zero will be such that $|p| = m|z|$. Instead of designing the parameters $K$, $m$ and $z_{PI}$ on the step response of the system, we will simulate the cart-pole with a push on the cart modeled as

\begin{equation*}
d(t) =
  \begin{cases}
    100\sin(\omega^\star t) & \text{if} \quad t\in[0, 0.25],\\
    0.0 & \text{otherwise.}
  \end{cases}
\end{equation*}
$\omega^\star = 4\pi \text{ rad/s}.$

## **Theoretical questions**
Before getting your hands on simulations, answer the following questions:

> ❓**Q1.** Let $P(s)$ and $C(s)$ denote the plant and controller transfer functions respectively. 
> 
> **a)** Draw the block diagram of the closed-loop system. Your diagram should include the following signals:
>  *   the reference $r(t)$,
>  *   the controlled input $u(t)$,
>  *   the controlled output $y(t)$,
>  *   the load disturbance $d(t)$,
>  *   the output noise $n(t)$.
> 
> **b)** Give the expression of $Y(s)$, the Laplace transform of the output signal, as a function of $R(s)$, $D(s)$ and $N(s)$, the Laplace transforms of $r(t)$, $d(t)$ and $n(t)$ respectively.

> ❓**Q2.** What do we call the *gang of 4* in control theory? Can you give the general expression and briefly explain what describe the complementary sensitivity function, the load disturbance sensitivity function, and the sensitivity function?
> (You may refer to the block diagram of the closed-loop system you drew in Q1.)


## **Controller design**

#### **The plant transfer function $P(s)$**

Before adding any control, let us first study the transfer function of the plant. 
> ❓**Q3.** Compute the poles and zeros of the plant transfer function $P(s)$. 
> Can you assess the stability of the system by using only the simplified Nyquist criterion? Why?

> ❓**Q4.** Without any control, is the system stable ? Justify your answer.
Below are the Bode and Nyquist plots of $P(s)$. It also shows how the output evolves in open-loop when the system experiences disturbance $d(t)$.

In [ ]:
fig = plt.figure("system_resp", figsize=(20,10), dpi=100, clear=True)
fig.subplots_adjust(hspace=0.3)
axes = fig.subplots(2, 2)
axes = axes.T.flatten()
bode_ax = axes[:2]
control.bode(TF_LIN_SYSTEM, omega_limits=[1e-2, 1e2], omega_num=1000, display_margins=True,
             ax=bode_ax, dB=True)
axes[0].set_title("Bode plots", fontsize='large', color='blue', fontweight='bold')

fig.sca(axes[2])
control.nyquist_plot(TF_LIN_SYSTEM, omega_limits=[1e-2, 1e2], omega_num=1000, ax=axes[2])
axes[2].set_title("Nyquist plot", fontsize='large', color='blue', fontweight='bold')

input = np.sin(4*np.pi*np.linspace(0, 15, 5000))*1.0
resp = control.forced_response(TF_LIN_SYSTEM, T=np.linspace(0, 15, 5000), U=input)
axes[3].plot(resp[0], resp[1])
axes[3].set_xlabel('Time [s]')
axes[3].set_ylabel('Amplitude [rad]')
axes[3].set_title("Disturbance response", fontsize='large', color='blue', fontweight='bold');
plt.show()
# display(plt.gcf())

#### **Lead compensator design**

Let us start with a simple lead compensator, i.e., $C(s)=K\frac{s+z}{s+p}$.

The cutoff frequency $\omega_c$ is the separation frequency between what is considered as disturbance ($\omega \ll \omega_c$) and what is considered as noise ($\omega \gg \omega_c$). 

For your design, you will choose the cutoff frequency $\omega_c$ such that $$\omega_c = 2\omega^\star,$$ where $\omega^\star$ is the frequency of the disturbance $d(t)$.

> ❓**Q5.** Choose the phase margin $\varphi_m$ such that a delay of $\tau = 20$ ms is acceptable.

> ❓**Q6.** We want the maximal increase in phase induced by the lead compensator to occur at the gain crossover frequency. From $\varphi_m$, compute the value of $m$, under the assumption that there is no PI block and $\angle P(j\omega_c) = -\pi$. Multiply the value you found by 2 so that an additional delay margin is added, with a small cost on performance. 
>
> (As a reminder, the variable $m$ defines the space between the pole and zero of the lead compensator, i.e., $|p|=m|z|$.)

> ❓**Q7.** Find the zero and pole locations so that the increase in phase margin induced by the lead compensator occurs at the gain crossover frequency.

> ❓**Q8.** Knowing that $|P(j\omega_c)| = -50.2808$ dB, give the value of $K$ such that $\omega_c$ stays the gain crossover frequency.

The cell below computes the Bode and Nyquist plots of the open-loop system, as well as the response of the closed-loop system to the disturbance $d(t)$.

In [ ]:
k_value = BoundedFloatText(value=1, min=0, max=10000, step=1, description='K')
zlead_value = BoundedFloatText(value=0, min=0, max=100, step=1, description='Zlead')
plead_value = BoundedFloatText(value=0, min=0, max=100, step=1, description='Plead')
update_but_lead = Button(description='Update')
lead_plot = FullPlot(fig_name='lead_plot')
update_but_lead.on_click(lead_plot._on_click)
lead_list = [k_value, zlead_value, plead_value, update_but_lead]
if COLAB:
    lead_list.append(lead_plot.output)
lead_vbox = VBox(lead_list, layout=Layout(flex_flow='column', align_items='center'))
out_lead = interactive_output(lead_plot._on_change, {'k': k_value, 'zlead': zlead_value, 'plead': plead_value})
display(lead_vbox)


#### **PI block design**

Now, we will add a PI block to the lead compensator. 
The transfer function of the controller is now $$C(s)=K\frac{s+z}{s+p}\frac{s+z_{PI}}{s}$$.

> ❓**Q9.** Use the simulation below to find a value for $z_{PI}$ that gives you a good closed-loop system response to the disturbance $d(t)$. Justify the positioning of $z_{PI}$ w.r.t. $\omega_c$.

In [ ]:
zpi = BoundedFloatText(value=1, min=0, max=10000, step=1, description='z_PI')
update_but_pi = Button(description='Update')
pi_plot = RespPlot(fig_name='pi_plot')
update_but_pi.on_click(pi_plot._on_click)
pi_list = [zpi, update_but_pi]
if COLAB:
    pi_list.append(pi_plot.output)
pi_vbox = VBox(pi_list, layout=Layout(flex_flow='column', align_items='center'))
out_pi = interactive_output(pi_plot._on_change, {'k': k_value, 'zlead': zlead_value, 'plead': plead_value, 'zpi' : zpi})
display(pi_vbox)

> ❓**Q10.** As far as the closed-loop response to disturbance is concerned, it seems that the PI block has added nothing to our design. The two cells below compute respectively the gang of four for the system controlled with a lead compensator and the system controlled with a lead compensator + PI block. Based on the comparison between both cases, comment on the usefulness of the PI block.

In [ ]:
update_gof = Button(description='Update')
gof_plot = GoFPlot(fig_name='gof_plot')
update_gof.on_click(gof_plot._on_click)
gof_list = [update_gof]
if COLAB:
    gof_list.append(gof_plot.output)
gof_vbox = VBox(gof_list, layout=Layout(flex_flow='column', align_items='center'))
out_gof = interactive_output(gof_plot._on_change, {'k': k_value, 'zlead': zlead_value, 'plead': plead_value, 'zpi': zpi})
display(gof_vbox)